<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-03-04 05:11:13--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-03-04 05:11:14--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  23.3MB/s    in 63s     

2024-03-04 05:12:17 (22.0 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [5]:
# method most_simmilar

'''your code'''

wv_embeddings.most_similar(positive=["dog"], topn=27)


[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885),
 ('canine', 0.7209057211875916),
 ('meow', 0.7185295820236206),
 ('cow', 0.7080444693565369),
 ('dog2', 0.7057910561561584),
 ('woof', 0.705061137676239),
 ('dog1', 0.7038840651512146),
 ('dog3', 0.701882004737854),
 ('penguin', 0.697029173374176),
 ('bulldog', 0.6940488815307617),
 ('mammals', 0.6931389570236206),
 ('bark', 0.6913798451423645),
 ('fruit', 0.6892251968383789),
 ('reptile', 0.6891209483146667),
 ('furry', 0.6863499283790588),
 ('carnivore', 0.6862949728965759),
 ('cat', 0.6852341294288635),
 ('horse', 0.6833381652832031)]

Ответ: слово cat не входит, но входит слово cats, 4-е место

слово cat на 26 месте

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [6]:
import numpy as np
import spacy
# import pymorphy2
import pandas as pd
import re
from nltk.tokenize import WordPunctTokenizer
from tqdm.notebook import tqdm
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

tokenizer = MyTokenizer()

In [7]:
# тестирование
tokenizer.tokenize('I live in Moscow')

['I', 'live', 'in', 'Moscow']

In [8]:
# """
#     question: строка
#     embeddings: наше векторное представление
#     dim: размер любого вектора в нашем представлении

#     return: векторное представление для вопроса

'''your code'''

def question_to_vec(question, embeddings, tokenizer, dim=200):

    result = []

    for word in tokenizer.tokenize(question.lower()):
        try:
        # if word in wv_embeddings:
            result.append(embeddings[word])
        except (KeyError):
            continue

    if len(result):
        result = np.mean(result, axis = 0)
    else:
        result = np.zeros(dim)
    return result

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [9]:
'''your code'''

q = 'I love neural networks '
vector = [question_to_vec(q, wv_embeddings, tokenizer, 200) for text in q ]
vector[0].shape

(200,)

In [10]:
vector = [question_to_vec(q, wv_embeddings, tokenizer, 200) for text in q ]
vector[0].shape

(200,)

In [11]:
vector[0][2].round(2)

-1.29

Ответ: -1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

Hits@47 = 1

DCG@1 = 0

Ответ: 1

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [18]:
(1/np.log2(1+9)*(9 <= 10)).round(1)

0.3

Ответ: 0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

Hits@K=1𝑁∑𝑖=1𝑁[𝑟𝑎𝑛𝑘_𝑞′𝑖≤𝐾],

In [12]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''

    hits_value = 1/len(dup_ranks)*sum([(rank <= k) for rank in dup_ranks])

    return hits_value

In [13]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dcg_value = 1/len(dup_ranks) * sum([(rank <= k) * 1/(np.log2(1 + rank)) for rank in dup_ranks])
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [14]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [16]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [17]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [18]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
        '''your code'''
    return data

Нам понадобиться только файл validation.

In [19]:
validation_data = read_corpus('/content/data/validation.tsv')
# train_data = read_corpus('/content/data/train.tsv')

Кол-во строк

In [20]:
validation_data[0]

['How to print a binary heap tree without recursion?',
 'How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname',
 'Use a usercontrol from another project to current webpage',
 '~ Paths resolved differently after upgrading to ASP.NET 4',
 'Materialize datepicker - Rendering when an icon is clicked',
 'Creating PyPi package - Could not find a version that satisfies the requirement iso8601',
 'How can I analyze a confusion matrix?',
 'How do I declare a C array in Swift?',
 'Using rand() when flipping a coin and rolling a die',
 'Handling a JSON field with a special character in its name in Java',
 'React Native select row on ListView when push it',
 "Get 'creation_time' of video using ffmpeg and regex",
 'Does row exist and multiple where',
 "How to specify a classifier in a gradle dependency's dependency?",
 'Using $unwind on multiple do

In [21]:
len(validation_data)

3760

Размер нескольких первых строк

In [22]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [24]:
'''your code'''

def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''

    pairs = []

    question_v = question_to_vec(question, embeddings, tokenizer, dim)
    # print(question_v)

    for i, el in enumerate(candidates):
        candidate_v =  question_to_vec(el, embeddings, tokenizer, dim)
        cos = cosine_similarity(question_v.reshape(1, -1), candidate_v.reshape(1, -1))
        pairs.append((i, el, cos[0][0]))

    pairs.sort(key=lambda c: c[2], reverse=True)

    return [el[:-1] for el in deepcopy(pairs)]


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [25]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [26]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [188]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

In [27]:
hide = ''
for r in ranks:
    print(r)
    hide += str(r[0])
print(hide)

(0, 'Getting all list items of an unordered list in PHP')
(2, 'select2 not displaying search results')
(1, 'WPF- How to update the changes in list item of a list')
021


ОТВЕТ: 021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [36]:
# функция для вычисления метрик ранга
def rank(tokenizer, mve):
    wv_ranking = []
    max_validation_examples = mve
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    return wv_ranking

In [37]:
# альтернативный токенайзер Spacy с нормализацией токенов
nlp = spacy.load('en_core_web_sm')

class SpacyT:
    def __init__(self):
        pass
    def tokenize(self, text):
        spacy_results = nlp(text)
        return [token.lemma_ for token in spacy_results]

In [35]:
res = {}

tokenizers = { MyTokenizer() : 'MyTokenizer', # разбиение на токены
              SpacyT() : 'Spacy', # применение нормализации токенов
               WordPunctTokenizer() : 'WordPunctTokenizer'} # токенизация с учетом пунктуации

res_df = pd.DataFrame()

for k, v in tokenizers.items():
    wv_ranking = rank(k, 10)
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        res[f'DCG@:{k}'] = dcg_score(wv_ranking, k)
        res [f'Hits@:{k}'] = hits_count(wv_ranking, k)
    res = dict(sorted(res.items(), key=lambda item: item[0]))
    df = pd.DataFrame.from_dict(res, orient='index', columns = [v])
    res_df = pd.concat((res_df, df), axis = 1)
res_df

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

,MyTokenizer,Spacy,WordPunctTokenizer
DCG@:1,0.200000,0.300000,0.200000
DCG@:10,0.522684,0.602715,0.494974
DCG@:100,0.543715,0.622539,0.538543
DCG@:1000,0.543715,0.622539,0.538543
DCG@:5,0.464871,0.539279,0.464871
DCG@:500,0.543715,0.622539,0.538543
Hits@:1,0.200000,0.300000,0.200000
Hits@:10,0.900000,0.900000,0.800000
Hits@:100,1.000000,1.000000,1.000000
Hits@:1000,1.000000,1.000000,1.000000


Вывод: применение нормализации (в данном случае через Spacy) улучшает результат, но очень сильно замедляет вычисление

### Эмбеддинги, обученные на корпусе похожих вопросов

In [38]:
train_data = read_corpus('./data/train.tsv')

In [39]:
train_data[:5]

[['converting string to list',
  'Convert Google results object (pure js) to Python object\n'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?\n'],
 ['Sending array via Ajax fails',
  'Getting all list items of an unordered list in PHP\n'],
 ['How to insert CookieCollection to CookieContainer?',
  'C# create cookie from string and send it\n'],
 ['Updating one element of a bound Observable collection',
  'WPF- How to update the changes in list item of a list\n']]

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [40]:
'''your code'''
pair = [p[0] + str(' ') + p[1] for p in train_data] # склеивание двух предложений
pair[:5]


['converting string to list Convert Google results object (pure js) to Python object\n',
 'Which HTML 5 Canvas Javascript to use for making an interactive drawing tool? Event handling for geometries in Three.js?\n',
 'Sending array via Ajax fails Getting all list items of an unordered list in PHP\n',
 'How to insert CookieCollection to CookieContainer? C# create cookie from string and send it\n',
 'Updating one element of a bound Observable collection WPF- How to update the changes in list item of a list\n']

In [41]:
words = [text.lower().split() for text in pair]
words[10]

['min',
 'value',
 'sql',
 'query',
 'row',
 'with',
 'minimum',
 'value',
 'of',
 'a',
 'column']

In [42]:
from gensim.models import Word2Vec

embeddings_trained = Word2Vec(words, # data for model to train on
                vector_size = 200,                 # embedding vector size
                min_count = 1,             # consider words that occured at least 5 times
                window = 5).wv

In [43]:
embeddings_trained.most_similar(positive=["python"], topn=5) # тест модели на выводе близких слов

[('python:', 0.8586578965187073),
 ('python?', 0.8092176914215088),
 ('python3', 0.8008866310119629),
 ('(python)', 0.7819085121154785),
 ('python,', 0.7568817138671875)]

In [44]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [46]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.274 | Hits@   1: 0.274
DCG@   5: 0.352 | Hits@   5: 0.423
DCG@  10: 0.375 | Hits@  10: 0.493
DCG@ 100: 0.430 | Hits@ 100: 0.759
DCG@ 500: 0.453 | Hits@ 500: 0.937
DCG@1000: 0.459 | Hits@1000: 1.000


In [47]:
# обернем расчет метрик в функцию, чтобы дальше использовать её в цикле для расчета по разным окнам
def rank_w2v(tokenizer):
    wv_ranking = []
    max_validation_examples = 10
    for i, line in enumerate(validation_data):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    return wv_ranking

In [48]:
# Пробуем разный размер окна  / cbow / min_count = 5

res_w2v = {}
res_df_w2v = pd.DataFrame()

window = [1, 2, 3, 4, 5]

tokenizer = MyTokenizer()

for w in window:
    embeddings_trained = Word2Vec(words, # data for model to train on
                    vector_size = 200,                 # embedding vector size
                    min_count = 1,             # consider words that occured at least 1 times
                    window = w).wv  # cbow
    wv_ranking = rank_w2v(tokenizer)

    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        res_w2v[f'DCG@:{k}'] = dcg_score(wv_ranking, k)
        res_w2v [f'Hits@:{k}'] = hits_count(wv_ranking, k)
    res = dict(sorted(res_w2v.items(), key=lambda item: item[0]))
    df = pd.DataFrame.from_dict(res, orient='index', columns = [f'window={w}'])
    res_df_w2v = pd.concat((res_df_w2v, df), axis = 1)

res_df_w2v

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

,window=1,window=2,window=3,window=4,window=5
DCG@:1,0.200000,0.200000,0.100000,0.100000,0.100000
DCG@:10,0.313093,0.324306,0.261649,0.279241,0.262991
DCG@:100,0.392271,0.363137,0.329436,0.307136,0.313052
DCG@:1000,0.414040,0.400768,0.352431,0.345673,0.351279
DCG@:5,0.313093,0.288685,0.200000,0.188685,0.227371
DCG@:500,0.403544,0.390046,0.341581,0.334789,0.340242
Hits@:1,0.200000,0.200000,0.100000,0.100000,0.100000
Hits@:10,0.400000,0.500000,0.500000,0.600000,0.500000
Hits@:100,0.800000,0.700000,0.800000,0.700000,0.700000
Hits@:1000,1.000000,1.000000,1.000000,1.000000,1.000000


In [49]:
# Пробуем разный размер окна / cbow / min_count = 5

res_w2v = {}
res_df_w2v = pd.DataFrame()

window = [1, 2, 3, 4, 5]

tokenizer = MyTokenizer()

for w in window:
    embeddings_trained = Word2Vec(words, # data for model to train on
                    vector_size = 200,                 # embedding vector size
                    min_count = 5,             # consider words that occured at least 5 times
                    window = w).wv  # cbow
    wv_ranking = rank_w2v(tokenizer)

    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        res_w2v[f'DCG@:{k}'] = dcg_score(wv_ranking, k)
        res_w2v [f'Hits@:{k}'] = hits_count(wv_ranking, k)
    res = dict(sorted(res_w2v.items(), key=lambda item: item[0]))
    df = pd.DataFrame.from_dict(res, orient='index', columns = [f'window={w}'])
    res_df_w2v = pd.concat((res_df_w2v, df), axis = 1)

res_df_w2v

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

,window=1,window=2,window=3,window=4,window=5
DCG@:1,0.200000,0.100000,0.100000,0.100000,0.100000
DCG@:10,0.313093,0.228906,0.300707,0.292125,0.279922
DCG@:100,0.389813,0.303671,0.326972,0.316590,0.303059
DCG@:1000,0.412042,0.341020,0.364346,0.353705,0.341156
DCG@:5,0.313093,0.200000,0.231753,0.193068,0.186135
DCG@:500,0.401044,0.341020,0.364346,0.353705,0.341156
Hits@:1,0.200000,0.100000,0.100000,0.100000,0.100000
Hits@:10,0.400000,0.400000,0.600000,0.600000,0.600000
Hits@:100,0.800000,0.700000,0.700000,0.700000,0.700000
Hits@:1000,1.000000,1.000000,1.000000,1.000000,1.000000


In [50]:
# Пробуем разный размер окна  / skip-gram / min_count = 5

res_w2v = {}
res_df_w2v = pd.DataFrame()

window = [1, 2, 3, 4, 5]

tokenizer = MyTokenizer()

for w in window:
    embeddings_trained = Word2Vec(words, # data for model to train on
                    vector_size = 200,                 # embedding vector size
                    min_count = 5,             # consider words that occured at least 5 times
                    window = w,
                    sg = 1).wv  # skip-gram
    wv_ranking = rank_w2v(tokenizer)

    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        res_w2v[f'DCG@:{k}'] = dcg_score(wv_ranking, k)
        res_w2v [f'Hits@:{k}'] = hits_count(wv_ranking, k)
    res = dict(sorted(res_w2v.items(), key=lambda item: item[0]))
    df = pd.DataFrame.from_dict(res, orient='index', columns = [f'window={w}'])
    res_df_w2v = pd.concat((res_df_w2v, df), axis = 1)

res_df_w2v

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

,window=1,window=2,window=3,window=4,window=5
DCG@:1,0.300000,0.200000,0.200000,0.200000,0.200000
DCG@:10,0.400000,0.437707,0.432562,0.467967,0.502372
DCG@:100,0.487615,0.455426,0.468298,0.501482,0.542115
DCG@:1000,0.497850,0.480833,0.479599,0.513241,0.554641
DCG@:5,0.400000,0.406161,0.399228,0.432347,0.502372
DCG@:500,0.487615,0.469981,0.479599,0.513241,0.554641
Hits@:1,0.300000,0.200000,0.200000,0.200000,0.200000
Hits@:10,0.500000,0.700000,0.700000,0.700000,0.700000
Hits@:100,0.900000,0.800000,0.900000,0.900000,0.900000
Hits@:1000,1.000000,1.000000,1.000000,1.000000,1.000000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

## Вывод:


Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?

В данном эксперименте лучше всего показал себя токенезатор Spacy с использованием лемм (нормальных форм), НО он работает гораздо медленнее более более порстых токенизаторов и потому проверить его работу удалось на малом клличестве примеров.

Если выбирать между токенизатором с применением регулярных выражений, убирающих понктуацию и токенизатором, учитывающим пунктуацию, то первый показал себя лучше, так как значи препинания в данном случае не сильно влияют на контекст из-за специфики текстов.

* Помогает ли нормализация слов?
Да, нормализация помогает.

* Какие эмбеддинги лучше справляются с задачей и почему?

Лучше всего справились с задачей эмбединги, созданные на основе Skip-Gram, с окном window = 5 и min_count = 5. Окно 5 позволило лучше определять конкстые слова по основному, видимо из-за особенностей текста, в котором ключевые слова встречаются в одном и том же окружении.


* Почему получилось плохое качество решения задачи?

Качество получено не очень хорошее из-за того, что w2v не запоминает контекст, технология устарела и имеет свои минусы.


* Предложите свой подход к решению задачи.

Использовать другие эмбеддинги, например, Glove, которые можно строить на n-граммах, что позволит лучше учитывать словоформы и соответственно контекст.

